In [56]:
import pandas as pd
from WOOF import WOOF
df = pd.read_excel("Columbia_Hackathon_Data_Dogfood.xlsx")

In [ ]:
# put products to correspond brands in a dictionary
df2 = pd.read_excel("cleaned_Dogfood.xlsx")
products = set(df2.product_name)
brands = set(df2.product_brand)
brand_products = {}
for brand in brands:
    brand_products[brand] = []
for product in products:
    clip = product.split()[0:2]
    clips = " ".join(clip)
    for brand in brands:
        if brand in clips:
            brand_products[brand].append(product)
            break
brand_products

In [65]:
def get_catergories(data, category): # input: "str"
        # return a list of catergories
        return list(set(data[category]))
    
def get_rank(data, col): # input: data = DataFrame, col = list["str"..]
        # return a list of the order of sns avg group by catergories increasing order
        rank = data.groupby(col).sns.mean().sort_values().index
        return list(rank)

def woof(data, brand, bio = None, ads_type = None):
        score = 0
        brand_score = get_rank(data, "product_brand").index(brand) / len(get_catergories(data, "product_brand"))
        return brand_score

In [66]:
df2 = pd.read_excel("cleaned_Dogfood.xlsx")
woof(df2, "Alpha")

0.44